<a href="https://colab.research.google.com/github/ismailsangu/IE423-TASK/blob/main/Task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# JOKE RECOMMENDATION:

## Initialize

In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [160]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [161]:
# Joke Id Data
dfJoke = pd.read_csv('/content/drive/MyDrive/Data_Sets/JokeText.csv')



# User ratings for each joke
dfRating1 = pd.read_csv('/content/drive/MyDrive/Data_Sets/UserRatings1.csv')
dfRating2 = pd.read_csv('/content/drive/MyDrive/Data_Sets/UserRatings2.csv')
dfRating = pd.concat([dfRating1, dfRating2], axis=1)


For more accurate result, user rating data sets are assembled.

In [162]:
dfJoke.head()

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [163]:
dfRating.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.64,4.32,6.99,-9.66,-8.4,-0.63,9.51,-7.67,-1.6,8.3


#RECOMMENDER SYSTEMS:

#1. Content Based Filtering

#Prepare Data

In [164]:
dfJoke.shape

(100, 2)

In [165]:
# Remove duplicates
dfJoke.drop_duplicates(subset ='JokeText', keep = 'first', inplace = True)
dfJoke.shape

(100, 2)

There is no duplicate.

In [166]:
dfJoke.isna().sum()

,0
JokeId,0
JokeText,0


There is no missed value.

In [167]:
dfRating.isna().any().any()

True

There are missed values.

In [168]:
dfRating =dfRating.fillna(method = 'ffill')

<ipython-input-168-07760ac53296>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dfRating =dfRating.fillna(method = 'ffill')


In [169]:
dfRating.isna().any().any()

True

There is still missed values.

In [170]:
cols_num_null = dfRating.columns[dfRating.isnull().any()].tolist()
dfRating = dfRating.drop(cols_num_null, axis=1)

In [171]:
dfRating.isna().any().any()


False

Missed values are dropped.

In [172]:
dfRating.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User63892,User64636,User64669,User65216,User65312,User66036,User67936,User68801,User69399,User70499
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,-0.87,-0.10,5.44,5.0,-2.77,-9.66,4.17,6.65,8.35,-5.92
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,-0.87,-0.10,5.44,5.0,-2.77,-9.66,4.17,8.20,4.71,-4.56
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,-0.87,-0.10,5.44,5.0,-2.77,-9.66,4.17,8.20,4.71,-4.56
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,-0.87,-0.10,5.44,5.0,-2.77,-9.66,4.17,8.20,4.71,-4.56
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,-4.42,2.57,-3.20,-9.9,-8.64,-8.98,1.26,-6.21,-6.50,-8.35


#Build Model

In [173]:
# Generate a matrix of common terms that show up in each joke
from sklearn.feature_extraction.text import TfidfVectorizer
mdlTfvJoke = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
tfidf_matrix = mdlTfvJoke.fit_transform(dfJoke['JokeText'])
tfidf_matrix.shape

(100, 3774)

In [174]:
# Calculate cosine similarity between each pair of movies as a function of the similarity of the common terms

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(100, 100)

#Predict

In [175]:
# Prepare recommendation function (build code from scratch and then package as function for ease of understanding)

titles = dfJoke['JokeText']
indices = pd.Series(dfJoke.index, index=dfJoke['JokeText'])

def get_recommendations(JokeText):
    idx = indices[JokeText]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    joke_indices = [i[0] for i in sim_scores]
    return titles.iloc[joke_indices]

In [176]:
def get_recommendations_by_keyword(keyword, top_n=10):

    # Create a query vector for the keyword
    keyword_vector = mdlTfvJoke.transform([keyword])

    # Calculate cosine similarity between the keyword and all jokes
    similarities = cosine_similarity(keyword_vector, tfidf_matrix)

    # Get indices of jokes sorted by similarity
    sorted_indices = similarities.argsort()[0][::-1]

    # Return the top_n most similar jokes
    return dfJoke['JokeText'].iloc[sorted_indices[:top_n]]


In [177]:
get_recommendations_by_keyword("engineer", top_n=5)

,JokeText
46,"There was an engineer who had an exceptional gift for fixing all \nthings mechanical. After serving his company loyally for over 30 \nyears, he happily retired. Several years later the company contacted \nhim regarding a seemingly impossible problem they were having with \none of their multi-million dollar machines. They had tried everything and \neveryone else to get the machine fixed, but to no avail. In \ndesperation, they called on the retired engineer who had solved so \nmany of their problems in the past.\nThe engineer reluctantly took the challenge. He spent a day studying \nthe huge machine. At the end of the day, he marked a small ""x"" in \nchalk on a particular component of the machine and proudly stated, \n""This is where your problem is"".\nThe part was replaced and the machine worked perfectly again. The \ncompany received a bill for $50,000 from the engineer for his \nservice.They demanded an itemized accounting of his charges. The \nengineer responded briefly:\nOne chalk mark $1 \nKnowing where to put it $49,999\nIt was paid in full and the engineer retired again in peace. \n"
27,"A mechanical, electrical and a software engineer from Microsoft were\ndriving through the desert when the car broke down. The mechanical\nengineer said ""It seems to be a problem with the fuel injection system,\nwhy don't we pop the hood and I'll take a look at it."" To which the\nelectrical engineer replied, ""No I think it's just a loose ground wire,\nI'll get out and take a look."" Then, the Microsoft engineer jumps in.\n""No, no, no. If we just close up all the windows, get out, wait a few\nminutes, get back in, and then reopen the windows everything will work\nfine.""\n"
48,"Three engineering students were gathered together discussing the\npossible designers of the human body. \nOne said, ""It was a mechanical engineer. Just look at all the joints."" \nAnother said, ""No, it was an electrical engineer. The nervous systems many thousands of electrical\nconnections."" \nThe last said, ""Actually it was a civil engineer. Who else would run a toxic waste pipeline through a recreational area?""\n"
92,"Reaching the end of a job interview, the human resources person asked a\nyoung engineer fresh out of Stanford,\n\n""And what starting salary were you looking for?""\n\nThe engineer said, ""In the neighborhood of $125,000 a year, depending\non the benefits package.""\n\nThe interviewer said, ""Well, what would you say to a package of 5-weeks \nvacation, 14 paid holidays, full medical and dental, company matching \nretirement fund to 50% of salary, and a company car leased every 2 years - \nsay, a red Corvette?""\n\nThe Engineer sat up straight and said, ""Wow! Are you kidding?""\n\nAnd the interviewer replied, ""Yeah, but you started it.""\n"
61,"A group of managers were given the assignment to measure the\nheight of a flagpole. So they go out to the flagpole with ladders\nand tape measures, and they're falling off the ladders, dropping\nthe tape measures - the whole thing is just a mess.\nAn engineer comes along and sees what they're trying to do,\nwalks over, pulls the flagpole out of the ground, lays it flat,\nmeasures it from end to end, gives the measurement to one of the\nmanagers and walks away.\nAfter the engineer has gone, one manager turns to another and\nlaughs. ""Isn't that just like an engineer, we're looking for the \nheight and he gives us the length.""\n"


In [178]:
get_recommendations_by_keyword("Turkish", top_n=5)

,JokeText
99,Q: Whats the difference between greeting a Queen and greeting the\nPresident of the United States?\n\nA: You only have to get on one knee to greet the queen.\n
36,"A Jewish young man was seeing a psychiatrist for an eating and\nsleeping disorder. \n\n""I am so obsessed with my mother... As soon as I go to sleep, I start\ndreaming, and everyone in my dream turns into my mother. I wake up in\nsuch a state, all I can do is go downstairs and eat a piece of toast.""\n\nThe psychiatrist replies:\n\n""What, just one piece of toast, for a big boy like you?""\n"
26,"Clinton returns from a vacation in Arkansas and walks down the\nsteps of Air Force One with two pigs under his arms. At the bottom\nof the steps, he says to the honor guardsman, ""These are genuine\nArkansas Razor-Back Hogs. I got this one for Chelsea and this one for\nHillary."" \n\nThe guardsman replies, ""Nice trade, Sir.""\n"
27,"A mechanical, electrical and a software engineer from Microsoft were\ndriving through the desert when the car broke down. The mechanical\nengineer said ""It seems to be a problem with the fuel injection system,\nwhy don't we pop the hood and I'll take a look at it."" To which the\nelectrical engineer replied, ""No I think it's just a loose ground wire,\nI'll get out and take a look."" Then, the Microsoft engineer jumps in.\n""No, no, no. If we just close up all the windows, get out, wait a few\nminutes, get back in, and then reopen the windows everything will work\nfine.""\n"
28,"An old Scotsmen is sitting with a younger Scottish gentleman and says the boy. \n""Ah, lad look out that window. You see that stone wall there, I built it with\nme own bare hands, placed every stone meself. But do they call me MacGregor the\nwall builder? No! \n\nHe Takes a few sips of his beer then says, ""Aye, and look out on that lake and \neye that beautiful pier. I built it meself, laid every board and hammered each\nnail but do they call me MacGregor the pier builder? No! \n\nHe continues...""And lad, you see that road? That too I build with me own bare \nhands. Laid every inch of pavement meself, but do they call MacGregor the road\nbuilder? No!""\n\nAgain he returns to his beer for a few sips, then says, \n""Agh, but you screw one sheep...""\n"


Thanks to Content Based Filtering,  we can take recommendation by keyword according to similarities even if the key word is not included.

### 2. Collaborative Filtering

#Prepare Data

In [179]:
dfRating.head(99)

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User63892,User64636,User64669,User65216,User65312,User66036,User67936,User68801,User69399,User70499
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,-0.87,-0.10,5.44,5.00,-2.77,-9.66,4.17,6.65,8.35,-5.92
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,-0.87,-0.10,5.44,5.00,-2.77,-9.66,4.17,8.20,4.71,-4.56
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,-0.87,-0.10,5.44,5.00,-2.77,-9.66,4.17,8.20,4.71,-4.56
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,-0.87,-0.10,5.44,5.00,-2.77,-9.66,4.17,8.20,4.71,-4.56
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,-4.42,2.57,-3.20,-9.90,-8.64,-8.98,1.26,-6.21,-6.50,-8.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,0.92,-1.75,-3.79,6.26,-3.45,3.74,4.17,1.21,3.74,...,-1.21,-7.72,5.15,-0.19,1.36,-9.61,-0.29,-0.78,4.32,4.51
95,95,6.31,-1.02,3.98,3.93,9.13,1.94,0.44,1.21,6.94,...,-1.21,-7.72,5.15,-0.19,1.36,-9.61,-0.29,-0.78,4.32,2.62
96,96,-4.95,-0.97,-6.46,-2.57,9.17,1.99,-0.78,5.34,5.83,...,-1.21,-7.72,5.15,-0.19,1.36,-9.61,-0.29,-0.78,4.32,2.62
97,97,-0.19,4.13,-6.89,1.07,9.17,3.45,-1.02,1.94,5.53,...,-1.21,6.89,5.15,-0.19,1.36,-9.61,-0.29,-0.78,4.32,2.62


In [180]:
dfRating.columns

Index(['JokeId', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7',
       'User8', 'User9',
       ...
       'User63892', 'User64636', 'User64669', 'User65216', 'User65312',
       'User66036', 'User67936', 'User68801', 'User69399', 'User70499'],
      dtype='object', length=32026)

In [181]:
# Assuming dfRating is your DataFrame
dfNRating = dfRating.transpose()
# Remove the first row (which becomes the header after transposing)
dfNRating = dfNRating.drop(dfNRating.index[0])
# Reset the column names after dropping the first row
dfNRating.columns = [f'JokeId{i-1}' for i in range(1, dfNRating.shape[1] + 1)]
# Reset index and rename it to start from 1 to the number of remaining rows
dfNRating.reset_index(drop=True, inplace=True)
dfNRating.index += 1
dfNRating['UserId'] = range(1, len(dfNRating) + 1)
# Ensure 'UserId' is the first column
dfNRating = dfNRating[['UserId'] + dfNRating.columns[:-1].tolist()]
# Assuming 'User' columns in dfRating are named like 'User1', 'User2', etc.
user_ids_in_dfRating = [int(col.split('User')[1]) for col in dfRating.columns if 'User' in col]

# Update 'UserId' column in dfNRating
dfNRating['UserId'] = user_ids_in_dfRating
# Display the resulting DataFrame
dfNRating.head(33000)



,UserId,JokeId0,JokeId1,JokeId2,JokeId3,JokeId4,JokeId5,JokeId6,JokeId7,JokeId8,...,JokeId90,JokeId91,JokeId92,JokeId93,JokeId94,JokeId95,JokeId96,JokeId97,JokeId98,JokeId99
1,1,5.10,4.90,1.75,-4.17,5.15,1.75,4.76,3.30,-2.57,...,5.34,-4.61,3.59,7.18,0.92,6.31,-4.95,-0.19,3.25,4.37
2,2,-8.79,-0.87,1.99,-4.61,5.39,-0.78,1.60,1.07,-8.69,...,3.59,1.21,2.86,-0.05,-1.75,-1.02,-0.97,4.13,-1.84,2.96
3,3,-3.50,-2.91,-2.18,-0.10,7.52,1.26,-5.39,1.50,-8.40,...,1.84,-4.03,-1.41,1.65,-3.79,3.98,-6.46,-6.89,-2.33,-7.38
4,4,7.14,-3.88,-3.06,0.05,6.26,6.65,-7.52,7.28,-5.15,...,-4.47,6.36,4.71,-5.19,6.26,3.93,-2.57,1.07,2.33,-0.34
5,5,-8.79,-0.58,-0.58,8.98,7.67,8.25,4.08,2.52,-9.66,...,-0.29,9.37,8.30,9.13,-3.45,9.13,9.17,9.17,9.08,8.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32020,66036,-9.66,-9.66,-9.66,-9.66,-8.98,-8.98,-4.42,-7.72,-7.72,...,-9.61,-9.61,-9.61,-9.61,-9.61,-9.61,-9.61,-9.61,-9.61,-9.61
32021,67936,4.17,4.17,4.17,4.17,1.26,1.26,-8.25,2.82,2.82,...,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29
32022,68801,6.65,8.20,8.20,8.20,-6.21,-6.21,-7.43,-0.53,-0.53,...,-0.78,-0.78,-0.78,-0.78,-0.78,-0.78,-0.78,-0.78,-0.78,-0.78
32023,69399,8.35,4.71,4.71,4.71,-6.50,-6.50,6.89,5.87,5.87,...,4.32,4.32,4.32,4.32,4.32,4.32,4.32,4.32,4.32,4.32


#Build Model

In [182]:
# Prepare data into Surprise library format

!pip3 install scikit-surprise #or !conda install -c conda-forge scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0,5))
# Melt the DataFrame to have three columns: UserId, JokeId, Rating
df_melted = dfNRating.melt(id_vars=['UserId'], var_name='JokeId', value_name='Rating')

# Load the data into Surprise format
X = Dataset.load_from_df(df_melted[['UserId', 'JokeId', 'Rating']], reader)
X_train, X_test = train_test_split(X, test_size=.25)

In [183]:
# Define SVD model

from surprise import SVD

mdlSvdRating = SVD()

In [184]:
# Fit SVD model

mdlSvdRating.fit(X_train)
test_pred = mdlSvdRating.test(X_test)

In [185]:
# Evalute SVD accuracy

from surprise import accuracy

accuracy.rmse(test_pred)

RMSE: 4.5411


4.541080950886233

RMSE: 4.5411 is quite high for a recommendation so we can improve accuracy by hyperparameter tuning.

In [ ]:
# Tune hyperparameters

from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(X)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# Cross-validate

from surprise.model_selection import cross_validate

cross_validate(mdlSvdRating, X, measures=['RMSE', 'MAE'], cv=3, verbose=True)

#Predict

In [ ]:
dfNRating[dfNRating['UserId'] == 1]

In [ ]:
mdlSvdRating.predict(1, 56)

#Conclusion



*   After the data cleaning,to make Content Based Filtering, we can take joke recommendation by keyword according to similarities even if the key word is not included by using TF-IDF Vectorization.
* To make Collaborative Filtering, data trained in SVD Model and made predictions of user joke rating according to other users ratings.

